In [15]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

# HW2:  Модель Word2vec
### Выполнили:  

* Булгаков Дмитрий

* Тефикова Алие

### Группа ИАД-2

# 0. Loading data

Составьте достаточно большую коллекцию текстов на любом языке или ис-пользуйте тексты отсюда:<br>
https://www.kaggle.com/c/word2vec-nlp-tutorial/data.

In [4]:
text_dataset = pd.read_csv('data/unlabeledTrainData.csv', sep=';', encoding = 'latin1')

In [14]:
def split_dataset_into_list_sentences(df):
    df_sentences_list = []
    for i in text_dataset.review.index:
        df_sentences_list.append(sent_tokenize(df.review.values[i]))
    return df_sentences_list

In [16]:
def split_sentences_list_into_words_list(sentence):
    return word_tokenize(sentence)

In [18]:
def flatten_list_of_lists(list_of_lists):
    return lambda list_of_lists: [item for sublist in list_of_lists for item in sublist]

In [22]:
def process_dataset(dataset):
    list_sentences = split_dataset_into_list_sentences(dataset)
    list_sentences = flatten_list_of_lists(list_sentences)
    return list_sentences

In [23]:
hh = process_dataset(text_dataset)

In [24]:
len(hh)

TypeError: object of type 'function' has no len()

# 1. Fitting Word2Vec model

Обучите модель word2vec. Оцените время обучения модели,используя модуль time. Есть два варианта обучения модели: по отзывам целиком и с уче-там границ предложений. В принципе, погрешностью, которая возникает в первом случае можно пренебречь, но если вы хотите учитывать границы предложений, то можно использовать sent_tokenize из nltk.tokenize

In [11]:
sent_tokenize_list = []
for i in text_dataset.review.index:
    sent_tokenize_list.append(sent_tokenize(text_dataset.review.values[i]))

In [50]:
%time
model = Word2Vec(sent_tokenize_list)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
